In [1]:
import pandas as pd
from google.cloud import storage
from io import BytesIO
import os
import re

### READ DATA FROM GCS

#### auto detect ingest date

In [2]:
def get_latest_ingest_date(bucket_name, dataset_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    prefix = f"raw/{dataset_name}/ingest_date="
    dates = {
        blob.name.split("ingest_date=")[1].split("/")[0]
        for blob in bucket.list_blobs(prefix=prefix)}
    return max(dates)

#### read raw data from gcs

In [3]:
def read_raw_from_gcs(
    bucket_name: str,
    dataset_name: str,
    ingest_date: str | None = None
):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    if ingest_date is None:
        ingest_date = get_latest_ingest_date(
            bucket_name, dataset_name)
    file_name = f"raw_{dataset_name}.csv"
    gcs_path = (
        f"raw/{dataset_name}/"
        f"ingest_date={ingest_date}/"
        f"{file_name}")
    blob = bucket.blob(gcs_path)
    data = blob.download_as_bytes()
    if data.startswith(b"<!DOCTYPE html"):
        raise ValueError("RAW file is HTML")
    df = pd.read_csv(BytesIO(data), engine="python")
    print(f"Loaded {dataset_name} (ingest_date={ingest_date})")
    return df

In [4]:
BUCKET_NAME = "fmcg-data-lake"

df_sales = read_raw_from_gcs(
    BUCKET_NAME, "sales_transaction")
df_inventory_outlet = read_raw_from_gcs(
    BUCKET_NAME, "inventory_outlet_daily")
df_inventory_warehouse = read_raw_from_gcs(
    BUCKET_NAME, "inventory_warehouse_daily")
df_outlet = read_raw_from_gcs(
    BUCKET_NAME, "outlet_master")
df_product = read_raw_from_gcs(
    BUCKET_NAME, "product_master")

Loaded sales_transaction (ingest_date=2026-01-20)
Loaded inventory_outlet_daily (ingest_date=2026-01-20)
Loaded inventory_warehouse_daily (ingest_date=2026-01-20)
Loaded outlet_master (ingest_date=2026-01-20)
Loaded product_master (ingest_date=2026-01-20)


### TRANSFORM/CLEANING

#### PRODUCT MASTER

##### EDA

In [5]:
def data_quality_overview(df: pd.DataFrame) -> pd.DataFrame:
    summary = pd.DataFrame({
        "data_type": df.dtypes,
        "total_rows": len(df),
        "null_count": df.isnull().sum(),
        "null_pct": (df.isnull().sum() / len(df) * 100).round(2),
        "unique_count": df.nunique()})
    return summary.sort_values("null_pct", ascending=False)

In [6]:
data_quality_overview(df_product)

,data_type,total_rows,null_count,null_pct,unique_count
PackSize_ml,object,45,11,24.44,3
SKU_CODE,object,45,0,0.00,45
product_name,object,45,0,0.00,15
Brand,object,45,0,0.00,3
category,object,45,0,0.00,1
sub_category,object,45,0,0.00,5


In [7]:
def value_counts_all_columns(df: pd.DataFrame):
    for col in df.columns:
        print(f"\n=== Value count: {col} ===")
        print(df[col].value_counts(dropna=False))

In [8]:
value_counts_all_columns(df_product)


=== Value count: SKU_CODE ===
SKU_CODE
FW001    1
FW002    1
FW003    1
SR004    1
SR005    1
SR006    1
TN007    1
TN008    1
TN009    1
MS010    1
MS011    1
MS012    1
SS013    1
SS014    1
SS015    1
FW016    1
FW017    1
FW018    1
SR019    1
SR020    1
SR021    1
TN022    1
TN023    1
TN024    1
MS025    1
MS026    1
MS027    1
SS028    1
SS029    1
SS030    1
FW031    1
FW032    1
FW033    1
SR034    1
SR035    1
SR036    1
TN037    1
TN038    1
TN039    1
MS040    1
MS041    1
MS042    1
SS043    1
SS044    1
SS045    1
Name: count, dtype: int64

=== Value count: product_name ===
product_name
Glowix Gentle Facial Wash          3
Glowix Vitamin C Serum             3
Glowix Hydrating Toner             3
Glowix Daily Moisturizer           3
Glowix SPF 50 Sunscreen            3
DermaPlus Acne Defense Cleanser    3
DermaPlus Retinol Night Serum      3
DermaPlus Calming Toner            3
DermaPlus Oil-Free Gel Cream       3
DermaPlus UV Shield SPF 30         3
Skinvera Bright Foam 

In [9]:
df_product[df_product.isnull().any(axis=1)]

,SKU_CODE,product_name,Brand,category,sub_category,PackSize_ml
3,SR004,Glowix Vitamin C Serum,Glowix,Skincare,Serum,NaN
6,TN007,Glowix Hydrating Toner,Glowix,Skincare,Toner,NaN
8,TN009,Glowix Hydrating Toner,Glowix,Skincare,Toner,NaN
10,MS011,Glowix Daily Moisturizer,Glowix,Skincare,Moisturizer,NaN
11,MS012,Glowix Daily Moisturizer,Glowix,Skincare,Moisturizer,NaN
14,SS015,Glowix SPF 50 Sunscreen,Glowix,Skincare,Sunscreen,NaN
21,TN022,DermaPlus Calming Toner,DermaPlus,Skincare,Toner,NaN
24,MS025,DermaPlus Oil-Free Gel Cream,DermaPlus,Skincare,Moisturizer,NaN
25,MS026,DermaPlus Oil-Free Gel Cream,DermaPlus,Skincare,Moisturizer,NaN
33,SR034,Skinvera Niacinamide Booster,Skinvera,Skincare,Serum,NaN


In [10]:
df_product

,SKU_CODE,product_name,Brand,category,sub_category,PackSize_ml
0,FW001,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,30 ml
1,FW002,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,50ml
2,FW003,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,100 ML
3,SR004,Glowix Vitamin C Serum,Glowix,Skincare,Serum,NaN
4,SR005,Glowix Vitamin C Serum,Glowix,Skincare,Serum,50ml
5,SR006,Glowix Vitamin C Serum,Glowix,Skincare,Serum,100 ML
6,TN007,Glowix Hydrating Toner,Glowix,Skincare,Toner,NaN
7,TN008,Glowix Hydrating Toner,Glowix,Skincare,Toner,50ml
8,TN009,Glowix Hydrating Toner,Glowix,Skincare,Toner,NaN
9,MS010,Glowix Daily Moisturizer,Glowix,Skincare,Moisturizer,30 ml


##### CLEANING/TRANSFORM

Standardisasi kolom & extract pack size

In [11]:
def extract_pack_size_ml(x):
    if pd.isna(x):
        return None
    match = re.search(r"\d+", str(x))
    return int(match.group()) if match else None

df_product["pack_size_ml"] = df_product["PackSize_ml"].apply(extract_pack_size_ml)

In [12]:
df_product[["PackSize_ml", "pack_size_ml"]].drop_duplicates()

,PackSize_ml,pack_size_ml
0,30 ml,30.0
1,50ml,50.0
2,100 ML,100.0
3,NaN,NaN


In [13]:
DEFAULT_PACK_SIZES = [30, 50, 100]

def fill_pack_size(df):
    df = df.copy()
    df["pack_size_source"] = "original"
    for product, group in df.groupby("product_name"):
        existing_sizes = sorted(
            group["pack_size_ml"].dropna().unique().tolist())
        missing_idx = group[group["pack_size_ml"].isna()].index
        available_sizes = [
            s for s in DEFAULT_PACK_SIZES
            if s not in existing_sizes]
        for i, idx in enumerate(missing_idx):
            if i < len(available_sizes):
                df.loc[idx, "pack_size_ml"] = available_sizes[i]
                df.loc[idx, "pack_size_source"] = "assumption"
    return df

In [14]:
df_product = fill_pack_size(df_product)
df_product.head()

,SKU_CODE,product_name,Brand,category,sub_category,PackSize_ml,pack_size_ml,pack_size_source
0,FW001,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,30 ml,30.0,original
1,FW002,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,50ml,50.0,original
2,FW003,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,100 ML,100.0,original
3,SR004,Glowix Vitamin C Serum,Glowix,Skincare,Serum,NaN,30.0,assumption
4,SR005,Glowix Vitamin C Serum,Glowix,Skincare,Serum,50ml,50.0,original


In [15]:
def standardize_product(df):
    df = df.copy()
    df = df.rename(columns={
        "SKU_CODE": "sku_id",
        "Brand ": "brand",
    })
    return df

In [16]:
df_product_clean = standardize_product(df_product)
df_product_clean.head()

,sku_id,product_name,brand,category,sub_category,PackSize_ml,pack_size_ml,pack_size_source
0,FW001,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,30 ml,30.0,original
1,FW002,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,50ml,50.0,original
2,FW003,Glowix Gentle Facial Wash,Glowix,Skincare,Cleanser,100 ML,100.0,original
3,SR004,Glowix Vitamin C Serum,Glowix,Skincare,Serum,NaN,30.0,assumption
4,SR005,Glowix Vitamin C Serum,Glowix,Skincare,Serum,50ml,50.0,original


#### OUTLET MASTER

##### EDA

In [17]:
data_quality_overview(df_outlet)

,data_type,total_rows,null_count,null_pct,unique_count
outlet_id,object,25,0,0.0,25
OutletName,object,25,0,0.0,25
CHANNEL,object,25,0,0.0,4
Region,object,25,0,0.0,4
city_name,object,25,0,0.0,4


In [18]:
value_counts_all_columns(df_outlet)


=== Value count: outlet_id ===
outlet_id
OUT-001    1
OUT-002    1
OUT-003    1
OUT-004    1
OUT-005    1
OUT-006    1
OUT-007    1
OUT-008    1
OUT-009    1
OUT-010    1
OUT-011    1
OUT-012    1
OUT-013    1
OUT-014    1
OUT-015    1
OUT-016    1
OUT-017    1
OUT-018    1
OUT-019    1
OUT-020    1
OUT-021    1
OUT-022    1
OUT-023    1
OUT-024    1
OUT-025    1
Name: count, dtype: int64

=== Value count: OutletName ===
OutletName
Guardian Sudirman               1
Toko Kosmetik Cantika Tebet     1
Guardian Dago                   1
Watsons Grand Indonesia         1
Toko Kosmetik Ayu Margonda      1
Guardian Bekasi Barat           1
Toko Kosmetik Indah Bogor       1
Watsons Central Park            1
Guardian Alam Sutera            1
Toko Kosmetik Ratu Karawaci     1
Guardian Tunjungan Plaza        1
Watsons Pakuwon Mall            1
Toko Kosmetik Sari Malang       1
Guardian Simpang Lima           1
Toko Kosmetik Putri Solo        1
Watsons Trans Studio Mall       1
Guardian Malioboro 

##### CLEANING/TRANSFORM

In [19]:
def transform_outlet_master(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    # standardize column names (strip spasi saja dulu)
    df.columns = df.columns.str.strip()
    # rename kolom ke snake_case
    df = df.rename(columns={
        "OutletName": "outlet_name",
        "CHANNEL": "channel",
        "Region": "region"})
    # standardize text fields
    df["outlet_name"] = df["outlet_name"].str.title()
    df["city_name"] = df["city_name"].str.title()
    df["region"] = df["region"].str.title()
    # normalize channel
    channel_map = {
        "gt": "GT",
        "modern trade" : "MT"}
    df["channel"] = (
        df["channel"]
        .str.lower()
        .map(channel_map)
        .fillna(df["channel"]))
    # deduplicate outlet_id
    df = df.drop_duplicates(subset=["outlet_id"])
    return df

In [21]:
df_outlet_clean = transform_outlet_master(df_outlet)
df_outlet_clean

,outlet_id,outlet_name,channel,region,city_name
0,OUT-001,Guardian Sudirman,MT,Jakarta Area,Bandung
1,OUT-002,Toko Kosmetik Cantika Tebet,GT,East Java,Surabaya
2,OUT-003,Guardian Dago,MT,East Java,Surabaya
3,OUT-004,Watsons Grand Indonesia,GT,Central Java,Surabaya
4,OUT-005,Toko Kosmetik Ayu Margonda,MT,Central Java,Jakarta
5,OUT-006,Guardian Bekasi Barat,MT,East Java,Surabaya
6,OUT-007,Toko Kosmetik Indah Bogor,MT,Jakarta Area,Jakarta
7,OUT-008,Watsons Central Park,MT,Jakarta Area,Bandung
8,OUT-009,Guardian Alam Sutera,GT,East Java,Jakarta
9,OUT-010,Toko Kosmetik Ratu Karawaci,MT,West Java,Jakarta


#### INVENTORY OUTLET DAILY

##### EDA

In [22]:
data_quality_overview(df_inventory_outlet)

,data_type,total_rows,null_count,null_pct,unique_count
end_stock,float64,101250,33650,33.23,31
stock_in,float64,101250,20471,20.22,4
snapshot_date,object,101250,0,0.00,90
outlet_id,object,101250,0,0.00,25
sku_code,object,101250,0,0.00,45


In [23]:
value_counts_all_columns(df_inventory_outlet)


=== Value count: snapshot_date ===
snapshot_date
2024-01-01    1125
2024-01-02    1125
2024-01-03    1125
2024-01-04    1125
2024-01-05    1125
              ... 
2024-03-26    1125
2024-03-27    1125
2024-03-28    1125
2024-03-29    1125
2024-03-30    1125
Name: count, Length: 90, dtype: int64

=== Value count: sku_code ===
sku_code
FW001    2250
FW002    2250
FW003    2250
SR004    2250
SR005    2250
SR006    2250
TN007    2250
TN008    2250
TN009    2250
MS010    2250
MS011    2250
MS012    2250
SS013    2250
SS014    2250
SS015    2250
FW016    2250
FW017    2250
FW018    2250
SR019    2250
SR020    2250
SR021    2250
TN022    2250
TN023    2250
TN024    2250
MS025    2250
MS026    2250
MS027    2250
SS028    2250
SS029    2250
SS030    2250
FW031    2250
FW032    2250
FW033    2250
SR034    2250
SR035    2250
SR036    2250
TN037    2250
TN038    2250
TN039    2250
MS040    2250
MS041    2250
MS042    2250
SS043    2250
SS044    2250
SS045    2250
Name: count, dtype: int64

=== Va

In [27]:
df_inventory_outlet[df_inventory_outlet.isnull().any(axis=1)]

,snapshot_date,sku_code,outlet_id,end_stock,stock_in
1,2024-01-01,FW001,OUT-002,NaN,10.0
3,2024-01-01,FW001,OUT-004,NaN,NaN
4,2024-01-01,FW001,OUT-005,NaN,NaN
5,2024-01-01,FW001,OUT-006,NaN,NaN
6,2024-01-01,FW001,OUT-007,6.0,NaN
...,...,...,...,...,...
101235,2024-03-30,SS045,OUT-011,NaN,10.0
101236,2024-03-30,SS045,OUT-012,NaN,NaN
101241,2024-03-30,SS045,OUT-017,0.0,NaN
101243,2024-03-30,SS045,OUT-019,0.0,NaN


##### CLEANING/TRANSFORM